<a href="https://colab.research.google.com/github/Levos06/AlmostGPT/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Levos06/AlmostGPT

Cloning into 'AlmostGPT'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 83 (delta 32), reused 71 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 36.17 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [2]:
%cd AlmostGPT

/content/AlmostGPT


In [5]:
!pip install datasets==2.18.0 fsspec==2024.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.2.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [4]:
!python train.py

Epoch 1, Loss: 5.9868
Модель сохранена по пути: /content/drive/MyDrive/my_transformer.pth
Epoch 2, Loss: 5.2384
Epoch 3, Loss: 4.9851
Epoch 4, Loss: 4.8296
Epoch 5, Loss: 4.7241
Epoch 6, Loss: 4.5607
Epoch 7, Loss: 4.3785
Epoch 8, Loss: 4.2046
Epoch 9, Loss: 3.9805
Epoch 10, Loss: 3.7912
Epoch 11, Loss: 3.5693
Модель сохранена по пути: /content/drive/MyDrive/my_transformer.pth
Epoch 12, Loss: 3.4322
Epoch 13, Loss: 3.2475
Epoch 14, Loss: 3.0476
Epoch 15, Loss: 2.8866
Epoch 16, Loss: 2.7135
Epoch 17, Loss: 2.5370
Epoch 18, Loss: 2.3926
Epoch 19, Loss: 2.2308
Epoch 20, Loss: 2.0696
Модель сохранена по пути: /content/drive/MyDrive/my_transformer.pth


In [5]:
import torch
from transformer import Transformer
from tokenizer import BPETokenizer
from utils import generate_src_mask, generate_tgt_mask

# Путь к сохранённой модели
PATH = '/content/drive/MyDrive/my_transformer.pth'

# Параметры модели
d_model = 512
num_heads = 8
d_ff = 2048
num_layers = 6
dropout = 0.1
max_len = 128
src_vocab_size = 219
tgt_vocab_size = 343

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка модели
model = Transformer(
    d_model=d_model,
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    max_len=max_len,
    d_ff=d_ff,
    num_heads=num_heads,
    num_layers=num_layers,
    dropout=dropout
).to(device)

model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

# Токенизаторы
tokenizer_src = BPETokenizer()
tokenizer_src.load_vocab("vocab_en.json")
tokenizer_tgt = BPETokenizer()
tokenizer_tgt.load_vocab("vocab_de.json")

sos_token = 1
eos_token = 2

def create_causal_mask(size):
    # Нижнетреугольная матрица [size x size] для tgt
    return torch.tril(torch.ones((size, size), dtype=torch.bool)).to(device)

def translate_sentence(sentence, max_len=40):
    model.eval()
    with torch.no_grad():
        src_tokens = tokenizer_src.encode(sentence)
        print(src_tokens)
        src_tensor = torch.tensor(src_tokens, dtype=torch.long).unsqueeze(0).to(device)  # [1, src_len]

        src_mask = torch.ones((1, 1, len(src_tokens), len(src_tokens)), dtype=torch.bool).to(device)

        tgt_input = torch.tensor([[sos_token]], dtype=torch.long).to(device)
        output_tokens = []

        for _ in range(max_len):
            tgt_mask = create_causal_mask(tgt_input.size(1)).unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_len, tgt_len]
            memory_mask = torch.ones((1, 1, tgt_input.size(1), src_tensor.size(1)), dtype=torch.bool).to(device)

            logits = model(
                src_tensor,
                tgt_input,
                src_mask=src_mask,
                tgt_mask=tgt_mask,
                memory_mask=memory_mask
            )
            #print(logits)

            next_token_id = logits.argmax(-1)[:, -1].item()
            if next_token_id == eos_token:
                break
            output_tokens.append(next_token_id)
            tgt_input = torch.cat([tgt_input, torch.tensor([[next_token_id]], device=device)], dim=1)

        return tokenizer_tgt.decode(output_tokens)

# Проверка
sentences = [
    "Hello, how are you?",
    "This is a test sentence.",
    "Translate this example."
]

for sent in sentences:
    print(f"Input: {sent}")
    print(f"Output: {translate_sentence(sent)}")
    print()


Input: Hello, how are you?
[38, 185, 69, 72, 162, 179, 80, 156, 82, 102, 30]
Output: Ein Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in 

Input: This is a test sentence.
[50, 200, 116, 58, 4, 77, 186, 99, 147, 71, 77, 157, 60, 62, 16]
Output: Ein Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in 

Input: Translate this example.
[50, 149, 71, 76, 109, 188, 92, 116, 62, 81, 58, 70, 73, 126, 16]
Output: Ein Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in Mann in 

